In [22]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime
from sagemaker import image_uris

sagemaker.__version__

'2.219.0'

In [25]:
version = "v1"
session = sagemaker.Session()
sm = session.sagemaker_client

experiment_name = "team1-index-predictor-model-evaluation"
trial_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
trial_name = f"model-evaluation-{version}-{trial_suffix}"

bucket_name = "team1-index-predictor-bucket"

data_input_prefix = "data/processed"
model_input_prefix = "models/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output"
output_bucket_prefix = "models/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/results"

processing_instance_type = "ml.t3.medium"
processing_instance_count = 1

In [26]:
with Run(
    experiment_name=experiment_name,
    run_name=trial_name,
    run_display_name=trial_name,
    sagemaker_session=session,
) as run:
    experiment_config = run.experiment_config

xgboost_container = image_uris.retrieve("xgboost", session.boto_region_name, version="1.3-1")

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

script_processor.run(
    code="../src/models/evaluator.py",
    inputs=[
        ProcessingInput(
            source=f"s3://{bucket_name}/{data_input_prefix}",
            destination="/opt/ml/processing/input",
        ),
        ProcessingInput(
            source=f"s3://{bucket_name}/{model_input_prefix}",
            destination="/opt/ml/processing/model",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=f"s3://{bucket_name}/{output_bucket_prefix}",
        )
    ],
    arguments=[
        "--input_path",
        f"/opt/ml/processing/input",
        "--data_version",
        version,
        "--target_column",
        "Close_target",
        "--model_path",
        "/opt/ml/processing/model/model",
        "--output_path",
        "/opt/ml/processing/output",
    ],
    experiment_config=experiment_config,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name sagemaker-xgboost-2024-06-19-08-13-06-971


....................................................................................
Evaluation report saved to /opt/ml/processing/output/evaluation_report.txt


In [35]:
!aws s3 ls s3://{bucket_name}/{output_bucket_prefix}/output/

In [38]:
!aws s3 cp s3://{bucket_name}/{output_bucket_prefix}/evaluation_report.txt .

download: s3://team1-index-predictor-bucket/models/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output/evaluation_report.txt to ./evaluation_report.txt
